In [38]:
from langchain.document_loaders import PyPDFDirectoryLoader

from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
import langchain_pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
loader=PyPDFDirectoryLoader("pdfs")

In [4]:
DataList=loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)

In [6]:
List=text_splitter.split_documents(DataList)

In [7]:
List[13].page_content

'End-to-end memory networks are based on a recurrent attention mechanism instead of sequence-\naligned recurrence and have been shown to perform well on simple-language question answering and\nlanguage modeling tasks [28].\nTo the best of our knowledge, however, the Transformer is the ﬁrst transduction model relying\nentirely on self-attention to compute representations of its input and output without using sequence-'

In [8]:
len(List)

76

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.getenv("OPENAI_API_KEY")

In [10]:
embeddings = OpenAIEmbeddings()
EmbededData=embeddings.embed_query("Hello world")
len(EmbededData)

C:\Users\gleme\AppData\Local\Temp\ipykernel_10068\2861471853.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


1536

In [11]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")


In [22]:
import pinecone

In [25]:
PC=pinecone.Pinecone(
    api_key=pinecone_api_key)

In [26]:
PC.Index("pinecone-testing-1")

In [27]:
docsearch=langchain_pinecone.Pinecone.from_texts(
    [doc.page_content for doc in List],embedding=embeddings,
    index_name="pinecone-testing-1")

In [28]:
print(docsearch)

In [30]:
docsearch.similarity_search("YOLOv7 outperforms which models")

[Document(id='304dbf0b-9cd8-4388-906a-ab739afeee66', metadata={}, page_content='be superior in quality while being more parallelizable and requiring signiﬁcantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-\nto-German translation task, improving over the existing best results, including\nensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,\nour model establishes a new single-model state-of-the-art BLEU score of 41.0 after\ntraining for 3.5 days on eight GPUs, a small fraction of the training costs of the'),
 Document(id='3a71b0e3-0443-4fbb-a28d-9a6488708cf8', metadata={}, page_content='the competitive models.\nOn the WMT 2014 English-to-French translation task, our big model achieves a BLEU score of 41.0,\noutperforming all of the previously published single models, at less than 1/4 the training cost of the\nprevious state-of-the-art model. The Transformer (big) model trained for English-to-French used\ndropout rate Pdrop = 0.1, i

In [31]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [54]:
from langchain_community.chat_models import ChatOpenAI
llm=ChatOpenAI(
    openai_api_key=openai_api_key,
    temperature=0.9,
    model_name="gpt-3.5-turbo")

C:\Users\gleme\AppData\Local\Temp\ipykernel_10068\631595585.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(


In [55]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever())

In [57]:
y=qa.run("What is the difference between YOLOv7 and YOLOv8?")

C:\Users\gleme\AppData\Local\Temp\ipykernel_10068\3507390869.py:1: RuntimeWarning: coroutine 'Chain.arun' was never awaited
  y=qa.run("What is the difference between YOLOv7 and YOLOv8?")


In [58]:
print(y)

I don't have information on YOLOv7 or YOLOv8 in the provided context. If you have specific details or references related to these versions, I can try to help further.


In [1]:
import sys


In [ ]:
while True:    
    user_input = input("Enter a question (or 'exit' to quit): ")
    if user_input.lower() == 'exit':
        print("Exiting the program.")
        sys.exit()
    if user_input.strip() == "":
        continue
    result= qa({'query': user_input})
    print(f"Answer: {result['result']}")